In [1]:
import os 
from glob import glob
import json
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import gc

In [2]:
no_usecols = [
    "txkey",
    "chid",
    "cano",
    "set",
]

In [17]:
data = pd.read_csv("/data2/ovary/hw_tmp/preprocessing_v2.csv")
keys = sorted(list(data.columns))

In [18]:
for i in ["mchno", "acqic", "mode_group_acqic", "mode_group_acqic_2", "mode_group_mchno", "mode_group_mchno_2"]:
    data[i] = data[i].astype("category")

In [19]:
test_df = data[data["set"] == "public_test"][data.columns.difference(no_usecols)]

In [20]:
models_list = glob(os.path.join("/data2/ovary/hw_tmp/previous_submit/", "*.json"))
thrs = json.load(open("credit_card_fraud_detection_2023/config/thr.json"))

In [21]:
submit = pd.DataFrame()
submit["label"] = test_df["label"]
for idx, model_path in enumerate(models_list):
    gc.collect()
    m = XGBClassifier()
    m.load_model(model_path)
    test_probs = m.predict_proba(test_df.drop("label", axis=1))
    test_preds = (test_probs[:, 1]>thrs[idx]).astype(int)
    submit[f"pred_{idx}"] = test_preds

/home/u/yenjia/miniconda3/envs/fintech/lib/python3.9/site-packages/xgboost/data.py:463: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  or is_sparse(dtype)
/home/u/yenjia/miniconda3/envs/fintech/lib/python3.9/site-packages/xgboost/data.py:464: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  or (is_categorical_dtype(dtype) and enable_categorical)
/home/u/yenjia/miniconda3/envs/fintech/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/u/yenjia/miniconda3/envs/fintech/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_

In [22]:
submit["pred"] = submit[[f"pred_{i}" for i in range(3)]].sum(axis=1)
submit["pred"] = (submit["pred"] >= 2).astype(int)
print(classification_report(submit["label"], submit["pred"]))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    606856
         1.0       0.99      0.92      0.95      2184

    accuracy                           1.00    609040
   macro avg       0.99      0.96      0.98    609040
weighted avg       1.00      1.00      1.00    609040

